# Imports

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Concatenate, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

from tensorflow.keras import backend as K
import gc

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

import torch
from diffusers import StableDiffusionPipeline

from tqdm import tqdm

# Parameters

In [2]:
# Stable diffusion models
sdms = [
    {"lib": "CompVis/", "name": "stable-diffusion-v1-1"},
    {"lib": "runwayml/", "name": "stable-diffusion-v1-5"},
    {"lib": "stabilityai/", "name": "stable-diffusion-2-1"}
]

In [3]:
temperatures = [5, 10, 15]

In [4]:
# Number of images to generate by stable diffusion (SD)
n_rows = 1000

In [5]:
# Directory to save images
os.makedirs("Generated Images", exist_ok=True) 

In [6]:
# NN Parameters
epochs = 25
batch_size = 16

In [7]:
# Distribution function house prices
with open('kde_price_function.pkl', 'rb') as f:
    kde_loaded = pickle.load(f)

# Loading data

In [8]:
work_df = pd.read_csv("houses_preprocessed.csv")
work_df.head()

,n_citi,bed,bath,sqft,price,image
0,-1.500387,-0.489366,-0.472771,-1.424855,228500,houses_preprocessed/1.jpg
1,-0.574868,-0.489366,-1.515838,-1.340002,273950,houses_preprocessed/2.jpg
2,-1.500387,-0.489366,-1.515838,-1.064963,350000,houses_preprocessed/3.jpg
3,-1.438092,0.477001,0.570296,0.363878,385100,houses_preprocessed/4.jpg
4,-1.500387,-0.489366,-1.515838,-1.064963,350000,houses_preprocessed/5.jpg


# Data Synthesis

In [9]:
# Iterate through models
for index, model in enumerate(sdms):
    print("Processing Model: " + model["name"] + " (Model " + str(index + 1) + "/" + str(len(sdms)) + ")")

    # Load the model with optimizations
    pipe = StableDiffusionPipeline.from_pretrained(model["lib"] + model["name"], 
                                                   torch_dtype=torch.float16,
                                                   use_safetensors=False)
    pipe.set_progress_bar_config(disable=True) # Disable progress bar per image
    pipe.to("cuda")
    
    # Enable memory optimizations
    pipe.enable_attention_slicing()

    # Iterate through temperature settings
    for temperature in temperatures:
        print(f"  Using temperature: {temperature}")
        
        # Set the temperature
        pipe.scheduler.config.temperature = temperature
        
        # Create output directory for this model-temperature combination
        model_output_dir = os.path.join("Generated Images", f"{model["name"]}-temperature-{temperature}")
        os.makedirs(model_output_dir, exist_ok=True)

        # Generate images
        for i in tqdm(range(n_rows), desc="Generating images"):
            # Sample a random price from the house price distribution (population)
            sampled_price = int(kde_loaded.resample(1)[0][0])
            
            # Create prompt with the sampled price
            current_prompt = f'''Generate an image of a house where the value of the house is ${sampled_price}.'''

            # Generate an image at height 320 and width 416 (must be divisible by 8)
            with torch.no_grad():  # <-- No gradients needed for image generation
                image = pipe(current_prompt, height=312, width=416).images[0]

            # Resize the image back to height 311 and width 415 (NN shapes)
            image_resized = image.resize((415, 311), Image.LANCZOS)

            # Save the image
            image_path = os.path.join(model_output_dir, f"{i}-{sampled_price}.jpg")
            image_resized.save(image_path)

Processing Model: stable-diffusion-v1-1 (Model 1/3)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  Using temperature: 5


Generating images: 100%|█████████████████████████████████████████████████████████| 1000/1000 [6:08:59<00:00, 22.14s/it]


  Using temperature: 10


...

Generating images: 100%|█████████████████████████████████████████████████████████| 1000/1000 [5:36:30<00:00, 20.19s/it]


  Using temperature: 10


Generating images: 100%|█████████████████████████████████████████████████████████| 1000/1000 [5:36:32<00:00, 20.19s/it]


  Using temperature: 15


Generating images: 100%|█████████████████████████████████████████████████████████| 1000/1000 [5:36:28<00:00, 20.19s/it]


# Experimental set up

## Train and Test the models on the same data partioning

In [10]:
# Split into X and y
X_work_df = work_df[work_df.columns.difference(['price'])]
y_work_df = work_df['price']

# Split work_df into train and test
# The train set will be the same length as the number of images that we generated using SD
X_train, X_test, y_train, y_test = train_test_split(X_work_df, y_work_df, train_size=n_rows, random_state=42)

In [11]:
# Train data
X_train_tab = X_train[['n_citi', 'bed', 'bath', 'sqft']].values 
X_train_img = X_train['image'] # pd

# Test data ith compression cols
X_test_tab = X_test[['n_citi', 'bed', 'bath', 'sqft']].values 
X_test_img = X_test['image'] # pd

# Print shapes
print("Training Data Shapes:")
print(f"Tabular features: {X_train_tab.shape}")
print(f"Image features: {X_train_img.shape}")
print(f"Target prices: {y_train.shape}")
print("\nTest Data Shapes:")
print(f"Tabular features: {X_test_tab.shape}")
print(f"Image features: {X_test_img.shape}")
print(f"Target prices: {y_test.shape}")

Training Data Shapes:
Tabular features: (1000, 4)
Image features: (1000,)
Target prices: (1000,)

Test Data Shapes:
Tabular features: (14297, 4)
Image features: (14297,)
Target prices: (14297,)


# Neural Networks and Models

## Base NN and Resnet

In [12]:
def base_nn():
    # Image processing branch
    img_input = Input(shape=(311, 415, 3), name='image_input')
    x = Conv2D(32, (3, 3), activation='relu')(img_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1)(x) # Regression output for price prediction

    # Define the model
    nn_model = Model(inputs=img_input, outputs=output)
    
    # Compile the model
    nn_model.compile(optimizer='adam',
                  loss='mae',
                  metrics=['mae', 'R2Score'])
    
    # Display model summary debug
    # nn_model.summary()

    return nn_model

In [13]:
def resnet_nn():
    # Image processing branch with pre-trained ResNet50
    res_net = ResNet50(weights='imagenet', include_top=False, input_shape=(311, 415, 3))
   
    # Unfreeze only the last 10 layers of resnet (fine-tuning) 
    res_net.trainable = False 
    for layer in res_net.layers[-10:]:
        layer.trainable = True

    # Image processing branch
    img_input = Input(shape=(311, 415, 3), name='image_input')
    x = res_net(img_input)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1)(x) # Regression output for price prediction
    
    # Define the model
    res_net_model = Model(inputs=img_input, outputs=output)
    
    # Compile the model
    res_net_model.compile(optimizer='adam', 
                          loss='mae',
                          metrics=['mae', 'R2Score'])
    
    # Display model summary debug
    # res_net_model.summary()

    return res_net_model

In [14]:
'''
I did not write this code, the code is from: https://www.tensorflow.org/tutorials/load_data/images
It helps us train the NN more dynamically, it loads images on the go, such that not all RAM is used up.
It does try to maximise RAM usage this is basically what the tf.data.AUTOTUNE does.
'''

# Loads an image and normalizes it from [0,1]
def process_example(image_path, label):
    # Load raw bytes and convert to RGB
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Normalize image to [0, 1] and convert to float32
    image = tf.image.convert_image_dtype(image, tf.float32)

    return image, label


# Creates on the fly data sets to train/test the model, we need this to not exceed memory
def create_dataset(image_paths, labels, shuffle=True):
    # Convert to tensors
    image_paths = tf.convert_to_tensor(image_paths)
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)

    # Build dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda img, lbl: process_example(img, lbl), num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset


def train_and_evaluate_nn(nn, 
                          X_train_img_paths, y_train,
                          X_test_img_paths, y_test,
                          verbose=1):

    # Dynamic dataset loading
    train_ds = create_dataset(X_train_img_paths, y_train, shuffle=True) # Shuffle to break ordering
    test_ds = create_dataset(X_test_img_paths, y_test, shuffle=False) # No shuffle, we arent learning, just predicting

    # Train and Test
    history = nn.fit(train_ds, epochs=epochs, verbose=verbose)
    test_loss, test_mae, r2 = nn.evaluate(test_ds, verbose=0)

    return history, test_loss, test_mae, r2

## Logistic regression

In [15]:
def train_and_evaluate_lin_model(model, X_train_tab, y_train, X_test_tab, y_test):
    # Train the model
    model.fit(X_train_tab, y_train)
    
    # Evaluate the model
    y_test_pred = model.predict(X_test_tab)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    
    return mae_test, r2

### Train and Evaluate

In [16]:
# Create NNs with tabular features = 4 (n_citi, bed, bath, sqft)
nn_base = base_nn()
nn_resnet = resnet_nn()
lin = LinearRegression()

In [17]:
# NN
print("Training Base NN")
nn_base_hist, _, nn_base_mae, nn_base_r2 = train_and_evaluate_nn(nn_base, X_train_img, y_train, X_test_img, y_test)
print(f"NN Base MAE: {nn_base_mae:.0f}\nNN Base R2: {nn_base_r2:.2f}")

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Base NN
Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - R2Score: -3.3274 - loss: 697340.5625 - mae: 697340.5625
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -1.2982 - loss: 474452.0000 - mae: 474452.0000
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.1523 - loss: 313049.6562 - mae: 313049.6562
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.1595 - loss: 298242.0312 - mae: 298242.0312
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.1827 - loss: 301307.9375 - mae: 301307.9375
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.1673 - loss: 315694.7188 - mae: 315694.7188
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.1765 - loss: 302511.5625 - mae: 302511.5625
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.1834 - loss: 313413.5938 - mae: 313413.5938
Epoch 9/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.1654 - loss: 304218.6250 - mae: 304218.6250
Epoch 10/25

0

In [18]:
# Resnet
print("Training Resnet")
nn_resnet_hist, _, nn_resnet_mae, nn_resnet_r2 = train_and_evaluate_nn(nn_resnet, X_train_img, y_train, X_test_img, y_test)
print(f"Resnet MAE: {nn_resnet_mae:.0f}\nResnet R2: {nn_resnet_r2:.2f}")

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Resnet
Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 166s 3s/step - R2Score: -3.6216 - loss: 715327.2500 - mae: 715327.2500
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - R2Score: -2.9789 - loss: 730437.1250 - mae: 730437.1250
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - R2Score: -3.1209 - loss: 703348.9375 - mae: 703348.9375
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - R2Score: -3.1398 - loss: 700492.5625 - mae: 700492.5625
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - R2Score: -3.3003 - loss: 685579.9375 - mae: 685579.9375
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - R2Score: -3.0054 - loss: 692966.5000 - mae: 692966.5000
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - R2Score: -2.8412 - loss: 663518.8125 - mae: 663518.8125
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - R2Score: -2.8379 - loss: 662327.7500 - mae: 662327.7500
Epoch 9/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - R2Score: -2.6693 - loss: 643796.4375 - mae: 643796.4375
Epo

0

In [19]:
# LR
print("Training LR")
lr_mae, lr_r2 = train_and_evaluate_lin_model(lin, X_train_tab, y_train, X_test_tab, y_test)
print(f"LR MAE: {lr_mae:.0f}\nLR R2: {lr_r2:.2f}")

Training LR
LR MAE: 223890
LR R2: 0.35


# Comparison NN vs DS NN

In [20]:
def load_ds_data(model, temp):
    # Load image paths and prices
    image_path = f"Generated Images/{model}-temperature-{temp}"
    X_train_ds = []
    y_train_ds = []
    
    for img_file in os.listdir(image_path):
        if img_file.endswith('.jpg'):
            # Load image
            image = image_path + "/" + img_file
            X_train_ds.append(image)
            
            # Extract price from filename (format: "i-price.jpg")
            price_str = img_file.split('.')[0].split('-')[1]  # Gets the price part
            y_train_ds.append(int(price_str))
    
    # Ensure loaded images match the expected number of generated images
    assert len(X_train_ds) == n_rows
    assert len(y_train_ds) == n_rows

    return X_train_ds, y_train_ds

In [21]:
def create_comparison(llm, model_results):
    # DF Structure
    comparison_data = {
        'Model': [],
        'MAE': [],
        'R2': []
    }
    
    for model_name, mae, r2 in model_results:
        comparison_data['Model'].append(model_name)
        comparison_data['MAE'].append(round(mae))
        comparison_data['R2'].append(round(r2, 3))
    
    # Make into df
    comparison_df = pd.DataFrame(comparison_data).set_index("Model")
    
    # Display df
    print(f"Comparison of Models for {llm}")
    display(comparison_df)
    print()
    return comparison_df

In [22]:
import logging

# Supress retracing warning
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # '0' = all messages, '3' = fatal only
tf.get_logger().setLevel(logging.ERROR)

In [23]:
# For each model
for index, model in enumerate(sdms):
    for temperature in temperatures:
        print(f"Processing Model: {model["name"]} (Model {index + 1}/{len(sdms)})")
    
        # Initialize a list to store results for each SD model
        model_results = []
    
        # Load synthetic data
        X_train_ds, y_train_ds = load_ds_data(model["name"], temperature)
        
        # Create, Train and Evaluate the models
        nn_base_ds = base_nn()
        nn_resnet_ds = resnet_nn()
        
        print("Training Base NN")
        _, _, nn_base_ds_mae, nn_base_ds_r2 = train_and_evaluate_nn(nn_base_ds, X_train_ds, y_train_ds, X_test_img, y_test)
        print("Training Resnet NN")
        _, _, nn_resnet_ds_mae, nn_resnet_ds_r2 = train_and_evaluate_nn(nn_resnet_ds, X_train_ds, y_train_ds, X_test_img, y_test)
        
        # Store results for both models
        model_results.append(("NN Base DS " + model["name"], nn_base_ds_mae, nn_base_ds_r2))
        model_results.append(("NN Resnet DS " + model["name"], nn_resnet_ds_mae, nn_resnet_ds_r2))
        
        # Store default results
        model_results.append(('NN Base', nn_base_mae, nn_base_r2))
        model_results.append(('NN Resnet', nn_resnet_mae, nn_resnet_r2))
        model_results.append(('LR (tabular only)', lr_mae, lr_r2))
    
        # Create a comparison of model results
        create_comparison(model["name"], model_results)
        
        # Try to clear NN from memory
        K.clear_session()
        gc.collect()

Processing Model: stable-diffusion-v1-1 (Model 1/3)
Training Base NN
Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - R2Score: -3.9015 - loss: 701388.9375 - mae: 701388.9375
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -1.7907 - loss: 456908.3750 - mae: 456908.3750
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.0820 - loss: 268319.5000 - mae: 268319.5000
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.0863 - loss: 280254.5625 - mae: 280254.5625
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.1106 - loss: 269112.2188 - mae: 269112.2188
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.1205 - loss: 270614.0938 - mae: 270614.0938
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.1563 - loss: 285455.9688 - mae: 285455.9688
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - R2Score: -0.1079 - loss: 275519.5938 - mae: 275519.5938
Epoch 9/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.099

,MAE,R2
Model,,
NN Base DS stable-diffusion-v1-1,293687,-0.104
NN Resnet DS stable-diffusion-v1-1,306722,-0.386
NN Base,279930,-0.134
NN Resnet,287067,0.034
LR (tabular only),223890,0.350



Processing Model: stable-diffusion-v1-1 (Model 1/3)
Training Base NN
Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - R2Score: -3.2399 - loss: 719873.3750 - mae: 719873.3750
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -1.6759 - loss: 504044.0938 - mae: 504044.0938
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - R2Score: -0.0920 - loss: 289847.4375 - mae: 289847.4375
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.0893 - loss: 291152.6562 - mae: 291152.6562
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.0895 - loss: 294902.3750 - mae: 294902.3750
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.1134 - loss: 285096.8125 - mae: 285096.8125
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.0729 - loss: 287636.4375 - mae: 287636.4375
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.1066 - loss: 287566.1250 - mae: 287566.1250
Epoch 9/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.11

,MAE,R2
Model,,
NN Base DS stable-diffusion-v1-1,287154,-0.101
NN Resnet DS stable-diffusion-v1-1,355742,-0.323
NN Base,279930,-0.134
NN Resnet,287067,0.034
LR (tabular only),223890,0.350



Processing Model: stable-diffusion-v1-1 (Model 1/3)
Training Base NN
Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - R2Score: -3.8262 - loss: 701291.6875 - mae: 701291.6875
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -1.8725 - loss: 476830.6562 - mae: 476830.6562
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.1246 - loss: 280709.5938 - mae: 280709.5938
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - R2Score: -0.0398 - loss: 266011.6562 - mae: 266011.6562
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.0792 - loss: 286689.6562 - mae: 286689.6562
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.1329 - loss: 293317.4688 - mae: 293317.4688
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.0414 - loss: 279629.9062 - mae: 279629.9062
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.0688 - loss: 271888.9688 - mae: 271888.9688
Epoch 9/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.06

...

,MAE,R2
Model,,
NN Base DS stable-diffusion-2-1,286136,-0.097
NN Resnet DS stable-diffusion-2-1,277520,-0.117
NN Base,279930,-0.134
NN Resnet,287067,0.034
LR (tabular only),223890,0.350



Processing Model: stable-diffusion-2-1 (Model 3/3)
Training Base NN
Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - R2Score: -3.1656 - loss: 702245.1875 - mae: 702245.1875
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -1.3087 - loss: 450475.8750 - mae: 450475.8750
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.0456 - loss: 288090.5625 - mae: 288090.5625
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.0603 - loss: 288082.7188 - mae: 288082.7188
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.0899 - loss: 292951.1250 - mae: 292951.1250
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.0928 - loss: 286964.7812 - mae: 286964.7812
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.0864 - loss: 291128.7500 - mae: 291128.7500
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.1200 - loss: 310753.2188 - mae: 310753.2188
Epoch 9/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - R2Score: -0.095

,MAE,R2
Model,,
NN Base DS stable-diffusion-2-1,281576,-0.148
NN Resnet DS stable-diffusion-2-1,408226,-1.139
NN Base,279930,-0.134
NN Resnet,287067,0.034
LR (tabular only),223890,0.350



Processing Model: stable-diffusion-2-1 (Model 3/3)
Training Base NN
Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - R2Score: -3.5141 - loss: 713651.1875 - mae: 713651.1875
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - R2Score: -1.2163 - loss: 429071.5000 - mae: 429071.5000
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.0985 - loss: 290272.2500 - mae: 290272.2500
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.0767 - loss: 286170.1875 - mae: 286170.1875
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - R2Score: -0.0981 - loss: 282180.6875 - mae: 282180.6875
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.1101 - loss: 287227.4688 - mae: 287227.4688
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - R2Score: -0.0938 - loss: 295982.7500 - mae: 295982.7500
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.1054 - loss: 284748.0938 - mae: 284748.0938
Epoch 9/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - R2Score: -0.077

,MAE,R2
Model,,
NN Base DS stable-diffusion-2-1,281204,-0.127
NN Resnet DS stable-diffusion-2-1,301273,-0.349
NN Base,279930,-0.134
NN Resnet,287067,0.034
LR (tabular only),223890,0.350
